##### Import Libraries

In [31]:
import pandas as pd
import numpy as np
from scipy import stats
from imblearn.over_sampling import SMOTE

##### Read datasets

In [32]:
CJ_releases_df = pd.read_csv('C:/MachineLearning/CJ_Releases_2022.csv')
CJ_receives_df = pd.read_csv('C:/MachineLearning/CJ_Receives_2022.csv')

### Phase I

In [33]:
# Column rename
CJ_releases_df.rename(columns={'Release Date': 'Date', 'Release Type': 'Type'}, inplace=True)
CJ_receives_df.rename(columns={'Receive Date': 'Date', 'Receive Type': 'Type'}, inplace=True)

CJ_releases_df['Record Type'] = 'Release'
CJ_receives_df['Record Type'] = 'Receive'


##### Datasets Integration

In [34]:

CriminalJustice_df = pd.concat([CJ_releases_df, CJ_receives_df], ignore_index=True)

file_path = 'C:\\CJ_ML\\CriminalJustice_df.csv'
CriminalJustice_df.to_csv(file_path, index=False)

In [35]:
# Define data type
data_types = CriminalJustice_df.dtypes

print(data_types)

Date                   object
Type                   object
Inmate Type            object
Gender                 object
Race                   object
Age                     int64
County                 object
Offense Code            int64
Offense                object
Offense Description    object
Sentence Date          object
Offense Date           object
Sentence (Years)       object
Record Type            object
dtype: object


In [36]:
# Unique values
print(CriminalJustice_df.nunique())

Date                    326
Type                     23
Inmate Type              42
Gender                    2
Race                      6
Age                      72
County                  251
Offense Code            716
Offense                   4
Offense Description     710
Sentence Date          5623
Offense Date           7783
Sentence (Years)         40
Record Type               2
dtype: int64


In [37]:
# Check for duplicate rows
duplicates = CriminalJustice_df[CriminalJustice_df.duplicated(keep='first')]

num_duplicates = duplicates.shape[0]
print(f"Number of duplicate rows: {num_duplicates}")

if num_duplicates > 0:
    print("Duplicate Rows:")
    print(duplicates)
    duplicate_indices = duplicates.index.tolist()
    print(f"Indices of duplicate rows: {duplicate_indices}")
else:
    print("No duplicate rows found.")


Number of duplicate rows: 1
Duplicate Rows:
             Date     Type Inmate Type Gender   Race  Age County  \
84000  08/01/2022  New Rcv          ID      M  Black   19  Falls   

       Offense Code  Offense        Offense Description Sentence Date  \
84000      13150005  Violent  AGG ASSLT W/DEADLY WEAPON    07/14/2022   

      Offense Date Sentence (Years) Record Type  
84000   08/26/2021         10 Years     Receive  
Indices of duplicate rows: [84000]


In [38]:

# Remove duplicate rows
CriminalJustice_df = CriminalJustice_df.drop_duplicates(keep='first')

duplicates_after_deletion = CriminalJustice_df[CriminalJustice_df.duplicated(keep='first')]
num_duplicates_after_deletion = duplicates_after_deletion.shape[0]

print(f"Number of duplicate rows after deletion: {num_duplicates_after_deletion}")
if num_duplicates_after_deletion > 0:
    print("There are still duplicate rows after deletion, which is unexpected.")
else:
    print("All duplicate rows have been successfully deleted.")

Number of duplicate rows after deletion: 0
All duplicate rows have been successfully deleted.


##### Handling missing values

In [39]:
# Null values
print(CriminalJustice_df.isnull().sum())

Date                      0
Type                      0
Inmate Type               0
Gender                    0
Race                      0
Age                       0
County                    0
Offense Code              0
Offense                   0
Offense Description     276
Sentence Date          8085
Offense Date              4
Sentence (Years)          0
Record Type               0
dtype: int64


In [40]:
# Fill missing  'Offense Description' based on the Offense Code
offense_code_to_description = CriminalJustice_df.dropna(subset=['Offense Description']).groupby('Offense Code')['Offense Description'].agg(pd.Series.mode).to_dict()
CriminalJustice_df.loc[CriminalJustice_df['Offense Description'].isnull(), 'Offense Description'] = CriminalJustice_df.loc[CriminalJustice_df['Offense Description'].isnull(), 'Offense Code'].map(offense_code_to_description)
offense_code_to_description = CriminalJustice_df.dropna(subset=['Offense Description']).groupby('Offense Code')['Offense Description'].agg(lambda x: pd.Series.mode(x).iloc[0]).to_dict()
CriminalJustice_df.loc[CriminalJustice_df['Offense Description'].isnull(), 'Offense Description'] = CriminalJustice_df.loc[CriminalJustice_df['Offense Description'].isnull(), 'Offense Code'].map(offense_code_to_description)

CriminalJustice_df['Offense Description'] = CriminalJustice_df['Offense Description'].fillna('Unknown')

In [41]:
# Fill missing 'Sentence Date' and 'Offense Date' with 'Unknown'
CriminalJustice_df['Sentence Date'] = CriminalJustice_df['Sentence Date'].fillna('Unknown')
CriminalJustice_df['Offense Date'] = CriminalJustice_df['Offense Date'].fillna('Unknown')

CriminalJustice_df.to_csv('C:\\CJ_ML\\CriminalJustice_df_filledDates.csv', index=False)

print(CriminalJustice_df.isnull().sum())

Date                   0
Type                   0
Inmate Type            0
Gender                 0
Race                   0
Age                    0
County                 0
Offense Code           0
Offense                0
Offense Description    0
Sentence Date          0
Offense Date           0
Sentence (Years)       0
Record Type            0
dtype: int64


In [42]:
# Aggregation
CriminalJustice_df['Offense Date'] = pd.to_datetime(CriminalJustice_df['Offense Date'], errors='coerce')
CriminalJustice_df['Sentence Date'] = pd.to_datetime(CriminalJustice_df['Sentence Date'], errors='coerce')

# Calculate the difference in days and create a new column 'Days Between'
CriminalJustice_df['Days Between'] = (CriminalJustice_df['Sentence Date'] - CriminalJustice_df['Offense Date']).dt.days
print(CriminalJustice_df['Days Between'])

0         118.0
1         624.0
2         170.0
3         219.0
4         468.0
          ...  
91593    1575.0
91594     717.0
91595     426.0
91596    2225.0
91597    1225.0
Name: Days Between, Length: 91597, dtype: float64


In [43]:
# Convert any non-numeric values to NaN and ensure numeric type
CriminalJustice_df['Age'] = pd.to_numeric(CriminalJustice_df['Age'], errors='coerce')
CriminalJustice_df['Days Between'] = pd.to_numeric(CriminalJustice_df['Days Between'], errors='coerce')

CriminalJustice_df.dropna(subset=['Age', 'Days Between'], inplace=True)

# Separate features and target variable
X = CriminalJustice_df[['Age', 'Days Between']]
y = CriminalJustice_df['Record Type'] 

# Apply SMOTE to balance the dataset
smote = SMOTE()
X_balanced, y_balanced = smote.fit_resample(X, y)

balanced_df = pd.concat([pd.DataFrame(X_balanced, columns=['Age', 'Days Between']), pd.DataFrame(y_balanced, columns=['Record Type'])], axis=1)



In [44]:
# Calculate Z-scores for 'Age' and 'Days Between' columns
CriminalJustice_df['Age_z_scores'] = stats.zscore(CriminalJustice_df['Age'])
CriminalJustice_df['Days_Between_z_scores'] = stats.zscore(CriminalJustice_df['Days Between']) 

threshold = 3

# Identify outliers for 'Age'
outliers_age = CriminalJustice_df[abs(CriminalJustice_df['Age_z_scores']) > threshold]
# Identify outliers for 'Days Between'
outliers_days_between = CriminalJustice_df[abs(CriminalJustice_df['Days_Between_z_scores']) > threshold]  

CriminalJustice_df = CriminalJustice_df[(abs(CriminalJustice_df['Age_z_scores']) <= threshold) & (abs(CriminalJustice_df['Days_Between_z_scores']) <= threshold)]  

CriminalJustice_df = CriminalJustice_df.drop(columns=['Age_z_scores', 'Days_Between_z_scores'])  

print("Outliers for 'Age':")
print(outliers_age)

print("\nOutliers for 'Days Between':")  
print(outliers_days_between)

print("\nDataset without outliers:")
print(CriminalJustice_df)

Outliers for 'Age':
             Date       Type Inmate Type Gender   Race  Age     County  \
0      09/14/2021     Parole          G2      M  Black   75     Travis   
1      05/13/2022  Discharge          G2      M  White   81  Galveston   
2      02/22/2022     Parole          MH      M  Black   83     Karnes   
3      01/31/2022        DMS          G1      M  White   76   Brazoria   
4      12/20/2021     Parole          G2      M  Black   75     Potter   
...           ...        ...         ...    ...    ...  ...        ...   
84829  07/07/2022    New Rcv          ID      M  White   73     Parker   
84956  08/10/2022    New Rcv          ID      F  White   81    Kaufman   
87348  12/10/2021    New Rcv          ID      M  White   73       Hays   
90713  12/03/2021    New Rcv          ID      M  White   78       Kerr   
91379  11/19/2021    New Rcv          ID      M  White   73     Upshur   

       Offense Code  Offense                             Offense Description  \
0          

In [58]:
# Aggregation
import re
from sklearn.preprocessing import MinMaxScaler

def sentence_to_numeric(sentence):
    if 'life' in sentence.lower().strip():
        return 100
    elif '+' in sentence:
        base_number = re.findall(r'\d+', sentence)
        return int(base_number[0]) if base_number else 0

    numbers = re.findall(r'\d+', sentence)

    if len(numbers) >= 2:
        lower, upper = map(int, numbers[:2])
        return (lower + upper) // 2
    elif numbers:
        return int(numbers[0])
    else:
        return 0

if 'Sentence (Years)' in CriminalJustice_df.columns:
    CriminalJustice_df['Sentence Numeric'] = CriminalJustice_df['Sentence (Years)'].apply(sentence_to_numeric)
else:
    print("'Sentence (Years)' column not found in the dataset.")

scaler = MinMaxScaler()
CriminalJustice_df['Sentence Scaled'] = scaler.fit_transform(CriminalJustice_df[['Sentence Numeric']].values.reshape(-1, 1))

print(CriminalJustice_df[['Sentence (Years)', 'Sentence Numeric', 'Sentence Scaled']].head())

   Sentence (Years)  Sentence Numeric  Sentence Scaled
34   2 Years & Less                 2             0.02
57   21 to 25 Years                23             0.23
59   11 to 15 Years                13             0.13
64   26 to 30 Years                28             0.28
65          3 Years                 3             0.03
